# 🤖 AI Chatbot with Memory (LangChain + OpenAI + Pinecone)
This notebook builds an **AI-powered chatbot** that:
- ✅ **Uses OpenAI's GPT model for intelligent responses**
- ✅ **Maintains memory using Pinecone vector storage**
- ✅ **Retrieves relevant context for dynamic conversations**
- ✅ **Can be extended with custom knowledge bases**

---

In [ ]:
# 📌 Install Dependencies
!pip install openai langchain pinecone-client dotenv

## 🔑 Set Up API Keys
You'll need API keys for:
- **OpenAI API** (for GPT responses)
- **Pinecone API** (for memory storage)

Run the following cell and enter your keys.

In [ ]:
import os
from dotenv import load_dotenv

# Load API keys from environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

# Ensure keys are provided
assert OPENAI_API_KEY, "❌ OpenAI API key missing! Set OPENAI_API_KEY in your environment."
assert PINECONE_API_KEY, "❌ Pinecone API key missing! Set PINECONE_API_KEY in your environment."

print("✅ API keys loaded successfully!")

## 🧠 Initialize Pinecone for Chatbot Memory
We'll use **Pinecone** to store and retrieve past chat interactions.

In [ ]:
import pinecone

# Initialize Pinecone
pinecone.init(api_key=PINECONE_API_KEY, environment="us-west1-gcp")

# Create a Pinecone index
index_name = "chatbot-memory"
if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=1536, metric="cosine")

# Connect to the index
index = pinecone.Index(index_name)
print("✅ Pinecone index initialized!")

## 🤖 Define Chatbot Function
This function will:
- Retrieve past chat history from Pinecone
- Use OpenAI's GPT model to generate responses
- Store new conversations in Pinecone
- Return a chatbot response

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI

# Initialize OpenAI chat model
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model_name="gpt-4")

def chatbot(query, user_id):
    """Handles chat queries with memory retrieval."""
    
    # Retrieve past conversations from Pinecone
    past_chats = index.query(user_id, top_k=3, include_metadata=True)
    chat_history = "\n".join([item["metadata"]["text"] for item in past_chats["matches"]])
    
    # Use GPT model to generate response
    chain = ConversationalRetrievalChain(llm=llm, retriever=None)
    response = chain.run(input=query, chat_history=chat_history)
    
    # Store the new interaction in Pinecone
    index.upsert(vectors=[(user_id, response, {"text": query})])
    
    return response

print("✅ Chatbot function ready!")

## 💬 Chat with AI
Test the chatbot by entering **user messages**.

In [ ]:
# Example conversation
user_id = "user123"

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("👋 Exiting chatbot.")
        break
    response = chatbot(user_input, user_id)
    print("AI:", response)

## 🎉 Done!
Your **AI chatbot with memory** is now running! 🚀🤖